In [42]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from datetime import timedelta
import pmdarima as pm
from pmdarima import model_selection;
import statsmodels.api as sm
import warnings
purchase_order_detail = pd.read_csv('purchaseOrderDetail.csv')
purchase_order_header = pd.read_csv('purchaseOrderHeader.csv')
bill_of_material = pd.read_csv('billOfMaterial.csv')
product_inventory = pd.read_csv('productInventory.csv')
product = pd.read_csv('product.csv')


In [33]:
steps = 1

In [43]:
def get_all_components(product_id, bom_data, multiplier=1, memo=None):
    if memo is None:
        memo = {}
    # If endDate is not null then it's no longer used.
    components = bom_data[
        (bom_data['ProductAssemblyID'] == product_id) & 
        (pd.isna(bom_data['EndDate']))
    ]
    
    if len(components) == 0:
        components = bom_data[bom_data['ProductAssemblyID'] == product_id]
        if len(components) == 0:
            return memo
    
    for _, row in components.iterrows():
        component_id = row['ComponentID']
        qty = row['PerAssemblyQty'] * multiplier
        memo[component_id] = memo.get(component_id, 0) + qty
        get_all_components(component_id, bom_data, qty, memo)
    return memo

data = pd.read_csv('SalesAndMake.csv')

purchase_order_detail = pd.merge(
    purchase_order_header,
    purchase_order_detail,
    on='PurchaseOrderID',
    how='inner'
)
purchase_order_detail['OrderDate'] = pd.to_datetime(purchase_order_detail['OrderDate'])

data['OrderDate'] = pd.to_datetime(data['OrderDate'])
data['Month'] = data['OrderDate'].dt.to_period('M')
monthly_sales = data.groupby(['Month', 'ProductID'])['OrderQty'].sum().reset_index()
last_order_date = data['OrderDate'].max()
predicted_sales = []
purchaseOrderIncrement = {}
last_purchase_date = purchase_order_detail['OrderDate'].max()
purchase_order_detail = purchase_order_detail[purchase_order_detail['OrderDate'] >= last_purchase_date - timedelta(days=30)]

for product_id in monthly_sales['ProductID'].unique():
    product_sales = monthly_sales[monthly_sales['ProductID'] == product_id].copy()
    
    # Convert Month period to datetime for comparison
    product_sales['OrderDate'] = product_sales['Month'].dt.to_timestamp()
    if product_sales['OrderDate'].max() < last_order_date - timedelta(days=30):
        continue
        
    product_sales = product_sales.set_index('Month').sort_index()
    product_sales.reset_index()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        try:
            model = sm.tsa.SARIMAX(endog=product_sales['OrderQty'], order=(2,1,1), disp=False)
            model_fit = model.fit(disp=False)
            forecast = model_fit.forecast(steps=steps)
            # Comment these lines to disable plotting
            # plt.figure(figsize=(12, 6))
            # plt.scatter(range(len(product_sales)), product_sales['OrderQty'], 
            #     color='blue', label='Historical Sales', marker='o')
            # plt.scatter(range(len(product_sales), len(product_sales) + 4), forecast.values, 
            #         color='red', label='Prediction', marker='x', s=100)
            # plt.title(f'Sales History and Prediction for Product {product_id}')
            # plt.xlabel('Time Period (Months)')
            # plt.ylabel('Order Quantity')
            # plt.legend()
            # plt.grid(True)
            # plt.show()
        except np.linalg.LinAlgError as e:
            print(f"LinAlgError for parameters p={2}, d={1}, q={1}: {e}")
        except Exception as e:
            print(f"Exception for parameters p={2}, d={1}, q={1}: {e}")
            
    last_two_months_sales = product_sales['OrderQty'].iloc[-2:]
    last_month_sales = last_two_months_sales.mean()
    precentageIncrease = forecast.values[0] / last_month_sales
    componentList = get_all_components(product_id, bill_of_material)
    for component_id, qty in componentList.items():
        if component_id not in purchaseOrderIncrement:
            purchaseOrderIncrement[component_id] = {'count': 1, 'total_increase': precentageIncrease}
        else:
            purchaseOrderIncrement[component_id]['count'] += 1
            purchaseOrderIncrement[component_id]['total_increase'] += precentageIncrease
    
    predicted_sales.append({
        'ProductID': product_id,
        'PredictedOrderQty': forecast.values[0],
        'LastMonthSales': product_sales['OrderQty'].iloc[-1],
        'PercentageIncrease': precentageIncrease * 100
    })
predicted_salesMake = predicted_sales

for component_id in componentList.keys():
    if component_id in purchaseOrderIncrement:
        avg_increase = purchaseOrderIncrement[component_id]['total_increase'] / purchaseOrderIncrement[component_id]['count']
        productPurchase = purchase_order_detail[purchase_order_detail['ProductID'] == component_id]
        # productPurchase = productPurchase[productPurchase['OrderDate'] >= last_order_date - timedelta(days=30)]
        purchaseOrderIncrement[component_id]['PredictedOrderQty'] = productPurchase['OrderQty'].sum() * avg_increase
        purchaseOrderIncrement[component_id]['LastMonthPurchase'] = productPurchase['OrderQty'].sum()
        purchaseOrderIncrement[component_id]['PercentageIncrease'] = avg_increase
    else:
        purchaseOrderIncrement[component_id]['PredictedOrderQty'] = 0
        purchaseOrderIncrement[component_id]['LastMonthPurchase'] = 0
        purchaseOrderIncrement[component_id]['PercentageIncrease'] = 0

print(purchaseOrderIncrement)

{324: {'count': 97, 'total_increase': 115.2032973869684, 'PredictedOrderQty': 0.0, 'LastMonthPurchase': 0, 'PercentageIncrease': 1.187662859659468}, 486: {'count': 97, 'total_increase': 115.2032973869684, 'PredictedOrderQty': 653.2145728127074, 'LastMonthPurchase': 550, 'PercentageIncrease': 1.187662859659468}, 325: {'count': 97, 'total_increase': 115.2032973869684, 'PredictedOrderQty': 8907.47144744601, 'LastMonthPurchase': 7500, 'PercentageIncrease': 1.187662859659468}, 326: {'count': 97, 'total_increase': 115.2032973869684, 'PredictedOrderQty': 8907.47144744601, 'LastMonthPurchase': 7500, 'PercentageIncrease': 1.187662859659468}, 327: {'count': 97, 'total_increase': 115.2032973869684, 'PredictedOrderQty': 0.0, 'LastMonthPurchase': 0, 'PercentageIncrease': 1.187662859659468}, 483: {'count': 97, 'total_increase': 115.2032973869684, 'PredictedOrderQty': 653.2145728127074, 'LastMonthPurchase': 550, 'PercentageIncrease': 1.187662859659468}, 399: {'count': 97, 'total_increase': 115.203297

In [44]:
other_sales = pd.read_csv('SalesAndNotMake.csv')

other_sales['OrderDate'] = pd.to_datetime(other_sales['OrderDate'])
other_sales['Month'] = other_sales['OrderDate'].dt.to_period('M')
monthly_sales = other_sales.groupby(['Month', 'ProductID'])['OrderQty'].sum().reset_index()
last_order_date = other_sales['OrderDate'].max()
predicted_sales = []

for product_id in monthly_sales['ProductID'].unique():
    product_sales = monthly_sales[monthly_sales['ProductID'] == product_id].copy()
    
    # Convert Month period to datetime for comparison
    product_sales['OrderDate'] = product_sales['Month'].dt.to_timestamp()
    if product_sales['OrderDate'].max() < last_order_date - timedelta(days=30):
        continue
        
    product_sales = product_sales.set_index('Month').sort_index()
    product_sales.reset_index()
    split_point = int(len(product_sales) * 0.8)
    train_data = product_sales.iloc[:split_point]
    test_data = product_sales.iloc[split_point:]
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        try:
            # print(f'p: {p}, d: {d}, q: {q}')
            model = sm.tsa.SARIMAX(endog=product_sales['OrderQty'], order=(2,1,1), disp=False)
            model_fit = model.fit(disp=False)
            forecast = model_fit.forecast(steps=1)
            # plt.figure(figsize=(12, 6))
            # plt.scatter(range(len(product_sales)), product_sales['OrderQty'], color='blue', marker='o', label='Actual Sales')
            # plt.scatter(range(len(product_sales), len(product_sales) + 4), forecast.values, color='red', marker='x', label='Predicted Sales')
            # plt.title(f'Sales Prediction vs Actual for Product {product_id}')
            # plt.xlabel('Time Period (Months)')
            # plt.ylabel('Order Quantity')
            # plt.legend()
            # plt.grid(True)
            # plt.show()
            # Comment these lines to disable plotting
        except np.linalg.LinAlgError as e:
            try:
                model = sm.tsa.SARIMAX(endog=product_sales['OrderQty'], order=(2,1,0), disp=False)
                model_fit = model.fit(disp=False)
                forecast = model_fit.forecast(steps=1)
            except:
                print(f"LinAlgError for parameters p={2}, d={1}, q={1}: {e}")
        except Exception as e:
            print(f"Exception for parameters p={2}, d={1}, q={1}: {e}")
            

    predicted_sales.append({
        'ProductID': product_id,
        'PredictedOrderQty': forecast.values[0],
        # Take 2 last month because the in store sales can be shipped at both start and end of month
        'LastMonthSales': product_sales['OrderQty'].iloc[-2:].mean(),
        'PercentageIncrease': forecast.values[0] / product_sales['OrderQty'].iloc[-2:].mean() * 100
    })
predicted_salesNotMake = predicted_sales


In [49]:
combinedTotalSales = []
for sales in predicted_salesMake:
    combinedTotalSales.append(sales)
for sales in predicted_salesNotMake:
    combinedTotalSales.append(sales)
    if sales['ProductID'] not in purchaseOrderIncrement:
        purchaseOrderIncrement[sales['ProductID']] = {'PredictedOrderQty': sales['PredictedOrderQty'],
                                                      'LastMonthPurchase': sales['LastMonthSales'],
                                                      'PercentageIncrease': sales['PercentageIncrease']}
    elif 'PredictedOrderQty' in purchaseOrderIncrement[sales['ProductID']]:
        purchaseOrderIncrement[sales['ProductID']]['PredictedOrderQty'] += sales['PredictedOrderQty'] 
        percentage_increase = purchaseOrderIncrement[sales['ProductID']]['PredictedOrderQty'] / \
                            purchaseOrderIncrement[sales['ProductID']]['LastMonthPurchase']
        purchaseOrderIncrement[sales['ProductID']]['PercentageIncrease'] = percentage_increase

combinedTotalSales_df = pd.DataFrame(combinedTotalSales)
combinedTotalSales_df.to_csv('predicted_sales.csv', index=False) 

formattedPurchaseOrderIncrement = [
    {
        'ProductID': component_id,
        'PredictedOrderQty': values['PredictedOrderQty'],
        'LastMonthPurchase': values['LastMonthPurchase'],
        'PercentageIncrease': values['PercentageIncrease'] * 100
    }for component_id, values in purchaseOrderIncrement.items() 
    if 'PredictedOrderQty' in values and values['PredictedOrderQty'] > 0
]

output_data = pd.DataFrame(formattedPurchaseOrderIncrement) 
output_data.to_csv('purchase_orders_prediction.csv', index=False)
